## RAG로 AI소믈리에 wine pairing

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI

## LLM을 통한 요리 이미지 -> 맛과 풍미 (image to text)

In [10]:
# image 에서 맛(풍미)에 대한 설명 text 생성
def describe_dish_flavor(query):
    # pass
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona: You are a highly skilled and perceptive culinary expert with a deep understanding of flavors, aromas, and textures in a wide variety of cuisines. Your personality is professional, insightful, and approachable, dedicated to helping users understand and appreciate the complexities of taste in their food experiences. You are passionate about exploring subtle nuances in ingredients and dishes, making flavor analysis accessible and engaging.

            Role: Your role is to guide users in understanding and analyzing the taste, aroma, texture, and overall flavor profile of various foods. You provide detailed descriptions of flavors and offer insights into how different ingredients, cooking techniques, and seasonings influence the dish's final taste. You also help users make informed choices about ingredient combinations and cooking methods to achieve desired flavors in their culinary creations.
            Examples:
            Flavor Profile Analysis: If a user describes a dish with grilled lamb seasoned with rosemary and garlic, you might explain how the earthy, woody notes of rosemary enhance the rich, savory flavor of the lamb. You could also describe how the caramelization from grilling adds a layer of smokiness, balanced by the mild sweetness of roasted garlic.
            Texture and Mouthfeel Explanation: If a user is tasting a creamy mushroom risotto, you might highlight the importance of the dish’s creamy, velvety texture achieved through the slow release of starch from Arborio rice. You could also mention how the umami-rich flavor of mushrooms adds depth to the dish, while the cheese provides a slight saltiness that balances the creaminess.
            Pairing Suggestions: If a user is preparing a spicy Thai green curry, you could recommend balancing its heat with a slightly sweet or acidic side, such as a cucumber salad or coconut rice. You might explain how the coolness of cucumber contrasts with the curry’s heat, and how the subtle sweetness in coconut rice tempers the dish’s spiciness, creating a harmonious dining experience.
         """),
        ("human", """
            이미지의 요리명과 풍미를 한 문장으로 요약해 주세요. 영어로 표현해 주세요.
        """)
    ])

    # image url list
    template = []

    if query.get("image_urls"):
        template += [{"image_url": image_url} for image_url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4095)

    chain = prompt | llm | StrOutputParser()

    return chain

In [11]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(describe_dish_flavor)

In [12]:
url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'
res = r1.invoke({
    "image_urls": [url]
})
print(res)

Grilled pork belly with kimchi offers a savory, smoky flavor complemented by the spicy, tangy notes of fermented cabbage.


- prompt : 전문적인 요리 사진을 얻을 수 있는 웹사이트를 알려줘

## 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
vector_store = PineconeVectorStore(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    embedding=embedding,
    namespace=os.getenv("PINECONE_NAMESPACE")
)

# query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."
query = "This dish is made with pork belly and kimchi, and the rich, juicy flavor of the pork belly combines with the spicy, sour flavor of the kimchi to create a harmonious taste."
vector_store.similarity_search(query, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

[Document(id='c9cc39e4-2c5b-4f2c-9db1-efab4bb08ebd', metadata={'row': 91624.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=': 91624\ncountry: US\ndescription: Very gamy and umami-rich aromas of bacon fat and animal hide meet with maple, boysenberry, white pepper, rosemary and teriyaki on the nose of this bottling. The savory and somewhat smoky touches thrive on the palate too, where roasted pork and charred plums combine with coffee bean and warm-toast flavors.\ndesignation: \npoints: 92\nprice: 60.0\nprovince: California\nregion_1: Ballard Canyon\nregion_2: Central Coast\ntaster_name: Matt Kettmann\ntaster_twitter_handle: @mattkettmann\ntitle: Kimsey 2014 Grenache (Ballard Canyon)\nvariety: Grenache\nwinery: Kimsey'),
 Document(id='e9a0cf7c-8e84-4382-b54f-f8a760bb2c12', metadata={'row': 125577.0, 'source': './wine_reviews/winemag-data-130k-v2.csv'}, page_content=": 125577\ncountry: Germany\ndescription: Spicy notes of cinnamon and sugar cookie lend warmth to thi

In [14]:
def search_wines(dish_flavor):
    embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))
    vector_store = PineconeVectorStore(
        index_name=os.getenv("PINECONE_INDEX_NAME"),
        embedding=embedding,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    results =  vector_store.similarity_search(dish_flavor, namespace=os.getenv("PINECONE_NAMESPACE"), k=5)

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

# query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."

# results = search_wines(query)

# print(results.dish_flavor)
# print(results.wine_reviews)

## RunnableLambda로 실행

In [15]:
r2 = RunnableLambda(search_wines)
query = "This dish is made with pork belly and kimchi, and the rich, juicy flavor of the pork belly combines with the spicy, sour flavor of the kimchi to create a harmonious taste."
# query = "이 요리는 삼겹살과 김치로, 고소하고 육즙이 풍부한 삼겹살의 풍미가 매콤하고 새콤한 김치와 어우러져 조화로운 맛을 냅니다."
res = r2.invoke(query)
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

This dish is made with pork belly and kimchi, and the rich, juicy flavor of the pork belly combines with the spicy, sour flavor of the kimchi to create a harmonious taste.
: 91624
country: US
description: Very gamy and umami-rich aromas of bacon fat and animal hide meet with maple, boysenberry, white pepper, rosemary and teriyaki on the nose of this bottling. The savory and somewhat smoky touches thrive on the palate too, where roasted pork and charred plums combine with coffee bean and warm-toast flavors.
designation: 
points: 92
price: 60.0
province: California
region_1: Ballard Canyon
region_2: Central Coast
taster_name: Matt Kettmann
taster_twitter_handle: @mattkettmann
title: Kimsey 2014 Grenache (Ballard Canyon)
variety: Grenache
winery: Kimsey
: 125577
country: Germany
description: Spicy notes of cinnamon and sugar cookie lend warmth to this juicy, sweet-tart blend of yellow cherry, peach, and apricot flavors. It's unabashedly fruit forward yet balanced neatly with zesty acidity

## r1, r2로 구성

In [16]:
img_url = 'https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg'
chain = r1 | r2
res = chain.invoke({
    "image_urls": [img_url]
})

In [17]:
print(res.get("dish_flavor"))
print(res.get("wine_reviews"))

Grilled pork belly with kimchi offers a savory, smoky flavor complemented by the spicy, tangy notes of fermented cabbage.
: 91624
country: US
description: Very gamy and umami-rich aromas of bacon fat and animal hide meet with maple, boysenberry, white pepper, rosemary and teriyaki on the nose of this bottling. The savory and somewhat smoky touches thrive on the palate too, where roasted pork and charred plums combine with coffee bean and warm-toast flavors.
designation: 
points: 92
price: 60.0
province: California
region_1: Ballard Canyon
region_2: Central Coast
taster_name: Matt Kettmann
taster_twitter_handle: @mattkettmann
title: Kimsey 2014 Grenache (Ballard Canyon)
variety: Grenache
winery: Kimsey
: 45075
country: New Zealand
description: A mouthfilling, off-dry Pinot Gris, Kim Crawford's 2007 continues his success with this variety. Pear and citrus notes finish clean, a refreshing blend of sugar and acid. Drink now.
designation: 
points: 87
price: 17.0
province: Marlborough
region

## 와인 추천

In [21]:
def recommand_wines(query):
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona: You are a refined and approachable virtual wine sommelier with a deep passion for wines, dedicated to helping users explore and enjoy the world of wine with confidence. Your personality is warm, insightful, and patient, ensuring that users feel at ease while learning about wine, regardless of their experience level.
            Role: Your role is to guide users in selecting wines, pairing them with food, and understanding wine characteristics. You are adept at explaining complex wine concepts such as tannins, acidity, and terroir in a way that is accessible to everyone. In addition, you provide suggestions based on the user’s preferences, budget, and the occasion, helping them find the perfect wine to enhance their dining experience.
            Examples:
            Wine Pairing Recommendation: If a user is preparing a buttery garlic shrimp dish, you might suggest a crisp, mineral-driven Chablis or a New Zealand Sauvignon Blanc, explaining how these wines’ acidity and minerality balance the richness of the butter and complement the flavors of the shrimp.
            Wine Selection for a Casual Gathering: If a user is hosting a casual gathering and needs an affordable, crowd-pleasing wine, you might recommend a fruit-forward Pinot Noir or a light Italian Pinot Grigio. Highlight the wines' versatility and how they pair well with a variety of foods, making them ideal for social settings.
            Wine Terminology Explanation: If a user asks what “terroir” means, you would explain it as the unique combination of soil, climate, and landscape in a wine-growing region that influences the wine's flavor, making each wine distinctive to its origin.
            """
         ),
        ("human", """
            와인 페어링 추천에 아래의 요리와 풍미, 와인 리뷰만을 참고하여 한글로 답변해 주세요.

            요리와 풍미:
            {dish_flavor}
         
            와인 리뷰:
            {wine_reviews}
         
            답변은 다음과 같은 값으로 json 데이터로 리턴해주세요. 결과를 한글로 번역해 주세요.
            recommend_wine :
            recommend_reason :
        """)
    ])
    
    llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4095)
    # chain = prompt | llm | StrOutputParser()
    chain = prompt | llm | JsonOutputParser()

    return chain


In [22]:
r1 = RunnableLambda(describe_dish_flavor)
r2 = RunnableLambda(search_wines)
r3 = RunnableLambda(recommand_wines)

chain = r1 | r2 | r3
img_urls = ['https://sitem.ssgcdn.com/95/55/96/item/1000346965595_i1_750.jpg']
res = chain.invoke({
    "image_urls": img_urls
})

In [23]:
print(res)

{'recommend_wine': 'Kimsey 2014 Grenache (Ballard Canyon)', 'recommend_reason': '이 와인은 베이컨 지방과 동물 가죽의 감칠맛이 풍부한 아로마와 함께 메이플, 보이즌베리, 화이트 페퍼, 로즈마리, 테리야키의 향을 제공합니다. 그릴에 구운 돼지고기와 잘 어울리는 감칠맛과 약간의 훈제된 터치가 돋보이며, 구운 돼지고기와 김치의 풍미를 잘 보완합니다.'}
